<a href="https://colab.research.google.com/github/ariahosseini/InterfacePhononsToolkit/blob/master/util/RayTracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Ray tracing code
---
A collection of functions used in the variouse photon and phonon ray tracing models


# Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

# System parameters

In [2]:
d = 1.1   # Length of the cell in x
L = 0.5   # Length in y
h = 2.0   # Depth of the box in z. We assume all rays start at the z = zero 
          # plane and head in the negative z direction
RadiusAtTop = 0.25  # Radius of cylinder or cone at its top
PitchAngle  = 2.0*np.pi/180.0   # If 0 = cylinder

# vectors for cone equation
co = np.array([0.0,0.0,0.0]) # Cone apex
cm = np.identity(3)          # Cone matrix
if PitchAngle == 0:
  # Cylinder
  r = RadiusAtTop
  s = 0.0
else:
  # Cone
  r = 0.0
  s = -(np.tan(PitchAngle))**2
  cone_height = RadiusAtTop/(np.tan(PitchAngle)) # Cone Height
  co[2] = cone_height # Center 

cm[2,2] = s

# Define the pillar parameters and walls of the cell

pillar = [co,cm]
walls  = [[[ 1, 0, 0],[ d, 0, 0]],
         [[ 0, 1, 0],[ 0, L, 0]],
         [[-1, 0, 0],[-d, 0, 0]],
         [[ 0,-1, 0],[ 0,-L, 0]],
         [[ 0, 0,-1],[ 0, 0,-h]],
         [[ 0, 0, 1],[ 0, 0, 0]]]
walls = np.array(walls)


Define the propabilities of each outcome when a ray hits a wall. There are three properties assosiated with a wall: 


*   The probability of transmission 
    p_abs   = 0.8  # Probability that a ray is absorbed by a wall if it is not transmitted
*   The Specularity: the probability that if the ray is reflected it does so speculalrly

In [3]:
def GetCumProbs(p_tran,spec,p_abs):
    """
    Compute the list of cumulative probabilities from the tree wall properties

    Input
      p_tran  : Probability of transmission 
      spec    : Specularity: the probability that if the ray is reflected it does so speculalrly
      p_abs   : Probability that a ray is absorbed by a wall if it is not transmitted

    Output
      cp_list : Cumulative probability list for transmission, absorbtion, specular 
                reflection and diffuse reflection 
    """
    p_absa  = (1.0-p_tran)*p_abs           # Absolute probability of absorbtion 
    p_rspec = (1.0-p_tran-p_absa)*(1-spec) # Absolute probability of specular reflection 
    p_rdiff = (1.0-p_tran-p_absa)*spec     # Absolute probability of diffuse reflection 

    p_list  = np.array([p_tran,p_absa,p_rspec,p_rdiff]) 
    cp_list = np.add.accumulate([p_tran,p_absa,p_rspec,p_rdiff]) 
    #print(p_list)
    #print(cp_list)
    return cp_list

wall_transmission_prob_list = [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]    
wall_absorbtion_prob_list   = [0.0, 0.0, 0.0, 0.0, 0.8, 0.8, 1.0]    
wall_specularity__list      = [0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 1.0]    

cumulative_probabilities = []

for i in range(len(wall_specularity__list)):
    p_tran = wall_transmission_prob_list[i]
    spec   = wall_absorbtion_prob_list[i]
    p_abs  = wall_specularity__list[i]
    cumulative_probabilities.append(GetCumProbs(p_tran,spec,p_abs))

cumulative_probabilities = np.array(cumulative_probabilities)
print(cumulative_probabilities)

[[1.   1.   1.   1.  ]
 [1.   1.   1.   1.  ]
 [1.   1.   1.   1.  ]
 [1.   1.   1.   1.  ]
 [0.   0.1  0.28 1.  ]
 [0.   0.1  0.28 1.  ]
 [0.   1.   1.   1.  ]]


# Auxilliary functions

Define a function to pick a random direction

In [4]:
def RandomDirection():
    """
       Generate a randomly oriented unit vector drawn from a uniform 
       distribution of the unit sphere
       
       input: 
       out:   random vector (numpy array)
    """      
    theta = np.arccos(1 - (2*np.random.uniform(0, 1)))   
    phi = np.random.uniform(-np.pi, np.pi)
    return np.array([np.sin(theta)*np.cos(phi),
                     np.sin(theta)*np.sin(phi),
                     np.cos(theta)])

def RandomDirectionInHalfSpace(nvec):
    """
       Generate a randomly oriented unit vector in the halfspace perpendicular 
       to the vector nv. This function is used to generate new diffusely 
       scattered ray directions from sccattering from a surface with normal 
       veror nvec

       input:
              nvec : unit vector (numpy array)
       out:   
              rdir : random vector (numpy array)
    """      
    theta = np.arccos(1 - (2*np.random.uniform(0, 1)))   
    phi = np.random.uniform(-np.pi, np.pi)
    rdir = np.array([np.sin(theta)*np.cos(phi),
                     np.sin(theta)*np.sin(phi),
                     np.cos(theta)])
    # Reflect the component of the random vector that is pointing out of the 
    # halfspace
    normalcomp = np.dot(rdir,nvec)
    rdir += (np.abs(normalcomp)-normalcomp)*nvec  
    return rdir


# Functions for finding wall intercepts

Define a finction to determine the distances to the walls of a cell 

In [5]:
WallDistance = lambda p, nv, wp, wn : np.dot(wp-p,wn)/np.dot(nv,wn)
SmallestPositiveValue = lambda l : min([i for i in l if i > 0])
IndexOfSmallestPositiveValue = lambda l : l.index(min([i for i in l if i > 0]))

Define a function to determine if a ray hits a pillar, and if the distance it has to travel to do so.

In [6]:
WallDistance = lambda p, nv, wp, wn : np.dot(wp-p,wn)/np.dot(nv,wn)
SmallestPositiveValue = lambda l : min([i for i in l if i > 0])
IndexOfSmallestPositiveValue = lambda l : l.index(min([i for i in l if i > 0]))

def DistanceToPillar(q,direct,pillar):
    """
       Follow a ray in a cell starting from point p traveling in direction nv 
       untill it eaither exits the cell or hits an object (the pillar) within 
       the cell.
        
       input:
              q         : ray starting position
              direct    : ray direction 
              pillar    : parameters needed to describe the pillar
       out:   
              distance  : Shortest positive distance slong ray to the pillar, 
                          or -1 if the distance is negative, or the ray misses 
                          the pillar
    """     

    unit = direct/np.linalg.norm(direct)
    [c,M] = pillar
    #c = pillar[0]
    #M = pillar[1]
    
    Chi_prime = q - c # vector to the pillar from the ray start point

    # Terms in the quadratic equation
    A = np.matmul(unit,np.matmul(M,unit))
    B = np.matmul(Chi_prime,np.matmul(M,unit))+np.matmul(unit,np.matmul(M,Chi_prime))
    C = np.matmul(Chi_prime,np.matmul(M,Chi_prime)) - r**2

    # Test if ray hits pillar
    Delta = B**2 - 4*A*C
    if Delta < 0:
        return -1.0 # Ray misses the pillar
    else:
        distance = [(-B + np.sqrt(Delta))/(2*A),
                    (-B - np.sqrt(Delta))/(2*A)]
        if max(distance) < 0: 
            return -1.0 # Ray has allready passed the pillar
        else:
            return SmallestPositiveValue(distance) # Hits the pillar

# Function to trace ray in a single cell 

Define a function that folows the flight of a ray in a cell untill it hits the walls or it hits the pillar

In [7]:
def PathInCell(p,nv,walls,pillar):
    """
       Follow a ray in a cell starting from point p traveling in direction nv 
       untill it eaither exits the cell or hits an object (the pillar) within 
       the cell.
        
       input:
              p         : starting position
              nv        : ray direction
              walls     : list of pairs of normal vector and reference positions 
                          used to define each of the walls that bound the cell 
              pillar    : parameters needed to describe the pillar
       out:   
              pn        : New position after flighht in cell
              wallindex : random vector (numpy array)
    """      

    # Find the distance along the ray to all the walls
    hit_dist = [WallDistance(p,nv,wp,wn) for wn,wp in walls]
    
    # Determine if the ray hits the pillar and append the distance to the 
    # pillar to the list of wall distances
    hit_dist.append(DistanceToPillar(p,nv,pillar)) 
    print(hit_dist)
    
    # Find the shortest positive distance to hit something -- 
    # and then retrun the index of the wall this corresponds to.
    wallindex = IndexOfSmallestPositiveValue(hit_dist)
    print("Ray hits Wall ",wallindex)
    
    # Determine the position that that the ray strikes the wall
    pn = p + hit_dist[wallindex]*nv
    
    return pn, wallindex

# Test 


#p  = RandomPositionInBox()
p = np.array([d,L,0])*0.5 # For debugging
nv = RandomDirectionInHalfSpace(np.array([0,0,-1]))
print(p)
print(nv)
print(PathInCell(p,nv,walls,pillar))


[0.55 0.25 0.  ]
[-0.39226083  0.52951161 -0.75216281]
[-1.4021282657045007, 0.47213317880556244, 4.206384797113501, -1.4163995364166873, 2.658998785307807, -0.0, -1.0]
Ray hits Wall  1
(array([ 0.36480065,  0.5       , -0.35512102]), 1)


# Function to trace full ray trajectory

Define a function that traces the trajectory of a ray through multiple cells and scattering events untill it is absorbed or lost through the top of the box.
Each time a ray hits a wall it can do one of four things:

1.   Stop (be adsorbed, or lost)
2.   Pass through unhindered
3.   Scatter diffusely
4.   Scatter speculalrly

There is s probability for each


In [8]:
def Transmit(nv):
    """
       Return the new direction and position of the ray after it is transmitted 
       point that is transmitted
    """
    return nv

def Adsorb(nv):
    """
       Return the new direction and position of the ray after it is transmitted 
       point that is transmitted
    """
    return nv

def Spec_reflect(nv,wallnormal):
    """
       Return the new direction and position of the ray after it is transmitted 
       point that is transmitted
    """
    nnv = nv - 2*np.dot(wallnormal,nv)*wallnormal
    return nnv

def Diff_reflect(nv,wallnormal):
    """
       Return the new direction and position of the ray after it is transmitted 
       point that is transmitted
    """
    nnv = RandomDirectionInHalfSpace(wallnormal)
    return nnv

def WallEvent(p,nv,wallnormal,cumulative_probabilities):
    """
       Decide what happens at a wall: The ray can either be:
        * Transmitted with its direction unchanged, but the ray point shifted
        * Adsorbed with the ray tracing stopped
        * Scattered specularly 
        * Scattered diffusely 
        Follow a ray through a sequesnce of cells and scattering events starting 
       from point p traveling in direction nv untill some stopping condition is 
       reached. Return the list of cell locations at which the ray changed 
       direction, and their corresponding wall index.
        
       input:
              po        : ray starting position
              nvo       : ray direction
              walls     : list of pairs of normal vector and reference positions 
                          used to define each of the walls that bound the cell 
              pillar    : parameters needed to describe the pillar
       out:   
              plist     : List of positions where scattering occured
              wallindex : coreesponding wallindex for these events
    """

    dice = np.random.rand()
    #print(dice)
    event = 0
    while dice > cumulative_probabilities[event]:
      event += 1
    print(event)

    # Switch between cases:
    switcher = {
        0: Transmit(nv),                    # Event 0: transmission
        1: Adsorb(nv),                      # Event 1: absorbtion 
        2: Spec_reflect(nv,wallnormal),     # Event 2: specular reflection
        3: Diff_reflect(nv,wallnormal),     # Event 3: diffuse reflection
    }
    nnv = switcher.get(event, "nothing")
    
    return event, nnv

print(p)
print(nv)
#print(walls)
wallindex = 4
wallnormal = walls[wallindex][0]
probabilities = cumulative_probabilities[wallindex]

WallEvent(p,nv,wallnormal,probabilities)

[0.55 0.25 0.  ]
[-0.39226083  0.52951161 -0.75216281]
3


(3, array([ 0.40224917, -0.8335701 , -0.37862448]))

In [9]:


def RayTrace(po,nvo,walls,pillar):
    """
       Follow a ray through a sequesnce of cells and scattering events starting 
       from point p traveling in direction nv untill some stopping condition is 
       reached. Return the list of cell locations at which the ray changed 
       direction, and their corresponding wall index.
        
       input:
              po        : ray starting position
              nvo       : ray direction
              walls     : list of pairs of normal vector and reference positions 
                          used to define each of the walls that bound the cell 
              pillar    : parameters needed to describe the pillar
       out:   
              plist     : List of positions where scattering occured
              wallindex : coreesponding wallindex for these events
    """
    p,nv1 = po,nvo # p is the initial starting point in the cell refference frame
    
    nwalls = walls.shape(walls)[0] # number of walls
    max_cells = 10                 # Max number of itterations
    step = 0
    distance = 0
    crow_vector = np.zeros(3)      # vector from starting point to current point 
                                   # as the crow flies

    # Initialize path and event recording
    path   = [p]
    events = [[5,6]] # Event 5 (birth) at wall 6 (the top) Only record events 
                     # where the ray changes direction.
    
    while step < max_cells:
        step += 1
        # Flight leg
        pn,wallindex = PathInCell(p,nv,walls,pillar) 
        crow_vector += pn-p
              
        # Scatter
        # If the ray hits the pillar we need to compute the normal vector at the 
        # point on the surface of the pillar where the ray hit
        if wallindex < nwalls: 
            wallnormal = PillarNormalAtPoint(np)
            cellshift  = np.zeros(3)
        else: 
            wallnormal = walls[wallindex,0] 
            cellshift  = -2*walls[wallindex,1]

        probabilities = cumulative_probabilities[wallindex] 
        scatterindex,nnv = WallEvent(p,nv,wallnormal,probabilities)
        
        # Set up new flight leg
        
        if scatterindex == 0:
            # If wallindex is for an external wall and the scatterindex is for 
            # transmit then the ray goes into the next cell over. In which case 
            # shift p for this cell, and then go onto the next leg
            p = np + cellshift
        else:
            # If wall index is for scattering then we continue in the current cell 
            # picking up a new leg starting from np and heading in the new direvtion 
            # nnv. In this case record the event, opdate nv and go onto the new leg.         
            p = np
            nv = nnv
            events.append([scatterindex,wallindex])
            path.append(p+crow_vector)
            if scatterindex==1:  
                # If scatterindex is for adsorb record the event and then stop.
            return path,events
              
    return path,events


# Test it
po = np.array([0.5,0.25,0.0])
nvo = np.array([0.0,0.0,-1.0])
path,events = RayTrace(po,nvo,walls,pillar)
print(path)
print(events)

IndentationError: ignored

Function for computing the ACF of a vector quantity 

In [10]:
def ACF(t_max, J):
    # Compute the autocorrelation of the instantaniouse flux vector 
    nd = J.shape
    time_intervals = nd[1]
    c = np.zeros([3,time_intervals*2])
    zpad = np.zeros(time_intervals)
    sf = t_max/float(time_intervals)
    for j in range(3):
        dft  = np.fft.fft(np.concatenate((J[j],zpad)))
        c[j] = np.fft.ifft(dft*np.conjugate(dft))*sf
    return c[:,:time_intervals]

Define a function that simulates a large number of rays and collects the ray impact data.

# Plotting functions

> Indented block

> Indented block





Set plot formatting style and define some plotting functions

In [12]:
# Define plotting functions

plt.rc("font", size = 18, family = 'sans-serif')
plt.rcParams["figure.figsize"] = (6, 5)
plt.rcParams['animation.html'] = 'html5'

def quickplot(x,y,xlab="x",ylab="y",plotlab=""):
    """
       Display a formatted plot of a single set of data
       
       input: x = x values
              y = y values
              e = uncertainty values
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x,y,'-')
    ax.set(title=plotlab,xlabel=xlab, ylabel=ylab)
    plt.show()
  
def quickploterr(x,y,e,col='',xlab="x",ylab="y",plotlab=""):
    """
       Display a formatted error bar plot of a single set of data
       
       input: x = x values
              y = y values
              e = uncertainty values
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x,y,'-'+col)
    ax.fill_between(x,y-e,y+e,alpha=0.2)
    ax.set(title=plotlab,xlabel=xlab, ylabel=ylab)
    #plt.savefig('test.pdf')  
    plt.show()

def quickmultiplot(data,xlab="x",ylab="y",plotlab=""):
    """
       Display a formatted plot of multiple sets of data
       
       input: data = List of data sets for plotting with each data set composed 
                     of 2 equal length vectors containing the x and y data 
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for x,y in data:
        ax.plot(x,y,'-')
        ax.set(title=plotlab,xlabel=xlab, ylabel=ylab)
    plt.show()
        
def quickmultierrplot(data,xlab="x",ylab="y",plotlab=""):
    """
       Display a formatted error bar plot of multiple sets of data
       
       input: data = List of data sets for plotting with each data set composed 
                     of 3 equal length vectors containing the x, y and uncertainty data 
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for x,y,e in data:
        ax.plot(x,y,'-')
        ax.fill_between(x,y-e,y+e,alpha=0.2)
        ax.set(title=plotlab,xlabel=xlab, ylabel=ylab)
    plt.show()